In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import random

In [3]:
# Загрузка данных
file_path = 'data/sampled_data_wb.csv'
df = pd.read_csv(file_path)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2434298 entries, 0 to 2434297
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   order_dt    object
 1   user_id     int64 
 2   nm_id       int64 
 3   subject_id  int64 
dtypes: int64(3), object(1)
memory usage: 74.3+ MB


In [6]:
# Очистка данных
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [7]:
# Подсчет количества покупок по категориям для каждого пользователя
user_category_counts = df.groupby(['user_id', 'subject_id']).size().reset_index(name='counts')

In [8]:
# Преобразование столбцов в категориальный тип данных для группированных данных
user_category_counts['user_id'] = user_category_counts['user_id'].astype('category')
user_category_counts['subject_id'] = user_category_counts['subject_id'].astype('category')

# Получение категориальных кодов для индексов строк и столбцов разреженной матрицы
rows = user_category_counts['user_id'].cat.codes
cols = user_category_counts['subject_id'].cat.codes
data = user_category_counts['counts'].values

# Удостоверимся, что все массивы имеют одинаковую длину
assert len(rows) == len(cols) == len(data), "Mismatch in lengths of the arrays"

In [9]:
from scipy.sparse import coo_matrix
# Создание разреженной матрицы
sparse_matrix = coo_matrix((data, (rows, cols)), shape=(user_category_counts['user_id'].cat.categories.size, user_category_counts['subject_id'].cat.categories.size))

In [10]:
# Кластеризация пользователей с использованием KMeans
kmeans = KMeans(n_clusters=100)
user_clusters = kmeans.fit_predict(sparse_matrix)

# user_clusters['user_id'] = user_clusters['user_id'].astype('category')
# user_clusters['subject_id'] = user_clusters['subject_id'].astype('category')
df['user_id'] = df['user_id'].astype('category')
df['subject_id'] = df['subject_id'].astype('category')
# Привязка кластера к каждому пользователю
df['cluster'] = user_clusters[df['user_id'].cat.codes]

In [11]:
df

,order_dt,user_id,nm_id,subject_id,cluster
0,2024-01-01,93887832,765682228,8916,17
1,2024-01-01,55238544,690882096,696,3
2,2024-01-01,82094328,676729164,576,23
3,2024-01-01,76104496,519119588,3036,61
4,2024-01-01,60436644,632703608,9116,89
...,...,...,...,...,...
2434292,2024-03-31,120588208,747376060,2584,2
2434293,2024-03-31,476158656,184127004,22104,20
2434294,2024-03-31,41842508,576226376,10024,55
2434295,2024-03-31,273388428,666005080,13988,3


In [12]:
# Получение расстояний до кластеров
cluster_distances = kmeans.transform(sparse_matrix)

In [13]:
# Получение топовых товаров в каждом кластере
def get_top_items_by_cluster(df, cluster_id, top_n=200):
    cluster_df = df[df['cluster'] == cluster_id]
    top_items = cluster_df['nm_id'].value_counts().head(top_n).index.tolist()
    return top_items



In [14]:
def recommend_items(user_id, df, top_n=200):
    user_cluster = df[df['user_id'] == user_id]['cluster'].values[0]
    
    # Получение расстояний до кластеров для текущего пользователя
    user_index = df['user_id'].cat.codes[df['user_id'] == user_id].values[0]
    distances = cluster_distances[user_index]
    
    # Сортировка кластеров по расстоянию
    nearest_clusters = np.argsort(distances)
    
    # Кластеры, которые пользователь уже покупал
    user_clusters = df[df['user_id'] == user_id]['cluster'].unique()
    
    # Фильтрация товаров, которые пользователь уже покупал
    purchased_items = set(df[df['user_id'] == user_id]['nm_id'].unique())
    
    # Получение топовых товаров из кластеров, которые пользователь уже покупал (70 товаров)
    top_items_from_user_clusters = []
    for cluster in user_clusters:
        items = get_top_items_by_cluster(df, cluster, top_n=200)
        top_items_from_user_clusters.extend([item for item in items if item not in purchased_items])
    random.shuffle(top_items_from_user_clusters)
    top_items_from_user_clusters = top_items_from_user_clusters[:150]

    # Получение топовых товаров из ближайших кластеров (70 товаров)
    top_items_from_nearest_clusters = []
    for cluster in nearest_clusters[1:10]:  # Например, 1-3 ближайшие кластеры
        items = get_top_items_by_cluster(df, cluster, top_n=200)
        top_items_from_nearest_clusters.extend([item for item in items if item not in purchased_items])
    random.shuffle(top_items_from_nearest_clusters)
    top_items_from_nearest_clusters = top_items_from_nearest_clusters[:30]

    # Получение топовых товаров из кластеров подальше (50 товаров)
    top_items_from_far_clusters = []
    for cluster in nearest_clusters[11:30]:  # Например, 4-6 кластеры по удаленности
        items = get_top_items_by_cluster(df, cluster, top_n=200)
        top_items_from_far_clusters.extend([item for item in items if item not in purchased_items])
    random.shuffle(top_items_from_far_clusters)
    top_items_from_far_clusters = top_items_from_far_clusters[:15]

    # Получение топовых товаров из совершенно далеких кластеров (10 товаров)
    top_items_from_distant_clusters = []
    for cluster in nearest_clusters[31:]:  # Например, кластеры 7 и далее
        items = get_top_items_by_cluster(df, cluster, top_n=200)
        top_items_from_distant_clusters.extend([item for item in items if item not in purchased_items])
    random.shuffle(top_items_from_distant_clusters)
    top_items_from_distant_clusters = top_items_from_distant_clusters[:5]

    # Объединение всех рекомендаций
    recommendations = (
        top_items_from_user_clusters +
        top_items_from_nearest_clusters +
        top_items_from_far_clusters +
        top_items_from_distant_clusters
    )

    return recommendations

In [15]:
# Сортировка данных по дате заказа для правильного временного разделения
df['order_dt'] = pd.to_datetime(df['order_dt'])
df.sort_values(by='order_dt', inplace=True)

# Разделение данных на тренировочные и тестовые выборки на основе даты
split_date = df['order_dt'].quantile(0.7)  # 70% данных для тренировки, 30% для тестирования
train_df = df[df['order_dt'] <= split_date]
test_df = df[df['order_dt'] > split_date]

# Фильтрация тестовой выборки, чтобы в ней были только те пользователи, которые есть в тренировочной выборке
train_users = train_df['user_id'].unique()
test_df = test_df[test_df['user_id'].isin(train_users)]

test_users = test_df['user_id'].unique()
test_purchases = test_df.groupby('user_id')['nm_id'].apply(list).to_dict()

C:\Users\donru\AppData\Local\Temp\ipykernel_19480\3762717338.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_purchases = test_df.groupby('user_id')['nm_id'].apply(list).to_dict()


In [16]:
import math

def precision_at_k(recommended_items, relevant_items, k=None):
    if k is None:
        k = len(recommended_items)
    recommended_items = recommended_items[:k]
    relevant_set = set(relevant_items)
    recommended_set = set(recommended_items)
    intersection = recommended_set.intersection(relevant_set)
    return len(intersection) / min(k, len(recommended_items))

def diversity_at_k(recommended_items, k=None):
    if k is None:
        k = len(recommended_items)
    if len(recommended_items) < 2:
        return 0
    recommended_items = recommended_items[:k]
    pairs = [(recommended_items[i], recommended_items[j]) for i in range(len(recommended_items)) for j in range(i + 1, len(recommended_items))]
    diversity_score = len(set(pairs)) / (k * (k - 1) / 2)
    return diversity_score

def ndcg_at_k(recommended_items, relevant_items, k=None):
    if k is None:
        k = len(recommended_items)
    recommended_items = recommended_items[:k]
    idcg = sum([1.0 / math.log2(i + 2) for i in range(min(len(relevant_items), k))])
    dcg = sum([1.0 / math.log2(i + 2) if recommended_items[i] in relevant_items else 0 for i in range(len(recommended_items))])
    return dcg / idcg if idcg > 0 else 0

def map_at_k(recommended_items, relevant_items, k=None):
    if k is None:
        k = len(recommended_items)
    recommended_items = recommended_items[:k]
    relevant_set = set(relevant_items)
    relevant_intersection = 0
    precision_sum = 0.0
    for i, item in enumerate(recommended_items):
        if item in relevant_set:
            relevant_intersection += 1
            precision_sum += relevant_intersection / (i + 1)
    return precision_sum / min(k, len(relevant_set)) if relevant_set else 0

def calculate_all_metrics(recommended_items, relevant_items, k=None):
    precision = precision_at_k(recommended_items, relevant_items, k)
    diversity = diversity_at_k(recommended_items, k)
    ndcg = ndcg_at_k(recommended_items, relevant_items, k)
    map_score = map_at_k(recommended_items, relevant_items, k)
    return {
        'precision': precision,
        'diversity': diversity,
        'ndcg': ndcg,
        'map': map_score
    }


In [17]:
import random

def evaluate_recommendations(test_users, test_purchases, train_df, num_users=100):
    selected_users = random.sample(list(test_users), min(num_users, len(test_users)))
    
    all_metrics = {
        'precision': [],
        'diversity': [],
        'ndcg': [],
        'map': []
    }

    for user_id in selected_users:
        recommended_items = recommend_items(user_id, train_df)
        relevant_items = test_purchases.get(user_id, [])
        
        metrics = calculate_all_metrics(recommended_items, relevant_items)
        all_metrics['precision'].append(metrics['precision'])
        all_metrics['diversity'].append(metrics['diversity'])
        all_metrics['ndcg'].append(metrics['ndcg'])
        all_metrics['map'].append(metrics['map'])

    average_metrics = {metric: np.mean(values) for metric, values in all_metrics.items()}

    return average_metrics



In [48]:
# Пример использования:
# test_users - список уникальных пользователей из тестового набора данных
# test_purchases - словарь, где ключи - user_id, а значения - список nm_id, купленных пользователем
# k - количество рекомендаций для расчета метрик
# num_users - количество пользователей для расчета метрик
average_metrics_results = evaluate_recommendations(test_users, test_purchases, train_df, num_users=100)

# Вывод средних значений метрик
print("Average Metrics for 100 users:")
print(f"Precision: {average_metrics_results['precision']:.4f}")
print(f"Diversity: {average_metrics_results['diversity']:.4f}")
print(f"NDCG: {average_metrics_results['ndcg']:.4f}")
print(f"MAP: {average_metrics_results['map']:.4f}")

Average Metrics for 100 users:
Precision: 0.0006
Diversity: 0.9658
NDCG: 0.0084
MAP: 0.0010


Average Metrics for 100 users:
Precision: 0.0006
Diversity: 0.9645
NDCG: 0.0166
MAP: 0.0075